In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import seaborn as sns

import time, random, requests
from bs4 import BeautifulSoup

In [2]:
#data = pd.read_excel("congress-trading-all.xlsx")
df = pd.read_csv("dataset_capitol-trades-scraper_2025-05-12_17-38-39-324.csv")

In [3]:
df.shape

(10153, 13)

In [4]:


# CLEAN  -----------------------------------------------------------
# unify case in the action column
df['type'] = df['type'].str.lower().str.strip()        # 'buy' / 'sell'

# convert traded date to datetime
df['trade_date'] = pd.to_datetime(df['traded'], dayfirst=True, errors='coerce')

df['price'] = df['price'].replace(['N/A', 'n/a', 'NA', 'na'], np.nan)

# clean price -> float  (drops $ and commas)
df['trade_price'] = (
    df['price']
      .str.replace(r'[\$,]', '', regex=True)
      .astype(float)
)

df = df.dropna(subset=['trade_price'])

# keep only rows that parsed correctly
df = df.dropna(subset=['trade_date', 'trade_price'])

# sort so merge / iteration is deterministic
df = df.sort_values(['politician_name',
                     'traded_issuer_ticker',
                     'trade_date'])

# MATCH  BUYS ↔ SELLS  ---------------------------------------------
pairs = []                    # will collect one dict per matched trade

for (member, ticker), g in df.groupby(['politician_name',
                                       'traded_issuer_ticker']):
    
    buys  = g[g['type'] == 'buy'].copy().reset_index(drop=True)
    sells = g[g['type'] == 'sell'].copy().reset_index(drop=True)
    
    # pointers for a simple one-to-one FIFO match
    i = j = 0
    while i < len(buys) and j < len(sells):
        buy_row  = buys.loc[i]
        
        # move j forward until we find a SELL *after* this BUY
        while j < len(sells) and sells.loc[j, 'trade_date'] <= buy_row['trade_date']:
            j += 1
        if j == len(sells):         # no more sells after this buy
            break
        
        sell_row = sells.loc[j]

        pairs.append({
            'politician_name'  : member,
            'ticker'           : ticker,
            'buy_date'         : buy_row['trade_date'],
            'sell_date'        : sell_row['trade_date'],
            'days_held'        : (sell_row['trade_date'] - buy_row['trade_date']).days,
            'buy_price'        : buy_row['trade_price'],
            'sell_price'       : sell_row['trade_price'],
            'size_buy'         : buy_row.get('size', np.nan),
            'size_sell'        : sell_row.get('size', np.nan),
            # simple % return (use midpoint of size band, exact shares, etc. if you have it)
            'pct_return'       : (sell_row['trade_price'] - buy_row['trade_price'])
                                 / buy_row['trade_price'] * 100
        })
        
        # advance both pointers for a one-to-one match
        i += 1
        j += 1

# RESULT  ----------------------------------------------------------
pair_df = pd.DataFrame(pairs)

print(f"Matched {len(pair_df):,} round-trip trades")
print(pair_df.head())

Matched 1,164 round-trip trades
   politician_name    ticker   buy_date  sell_date  days_held  buy_price  \
0     Ashley Moody    HWM:US 2025-01-22 2025-02-24         33     126.71   
1  Bruce Westerman   AAPL:US 2025-03-03 2025-03-20         17     238.03   
2  Bruce Westerman   AMZN:US 2025-03-03 2025-03-20         17     205.02   
3  Bruce Westerman   AVGO:US 2025-03-03 2025-03-20         17     187.37   
4  Bruce Westerman  GOOGL:US 2025-03-03 2025-03-20         17     167.01   

   sell_price  size_buy size_sell  pct_return  
0      129.56  50K–100K  50K–100K    2.249231  
1      214.10   15K–50K    1K–15K  -10.053355  
2      194.95   15K–50K    1K–15K   -4.911716  
3      190.54    1K–15K    1K–15K    1.691840  
4      162.80   15K–50K    1K–15K   -2.520807  


In [5]:
unique_tickers = sorted(df['traded_issuer_ticker'].dropna().unique().tolist())
print(unique_tickers)

['$AERO', '$BTC', '$ETH', '$GRT', '$SKI', '$SOL', '$VELO', '$VIRTUAL', '$XRP', 'A:US', 'AA:US', 'AAGIY:US', 'AAL:US', 'AAON:US', 'AAPL:US', 'AAXJ:US', 'ABB:US', 'ABBV:US', 'ABCB:US', 'ABEV:US', 'ABNB:US', 'ABT:US', 'ACGL:US', 'ACI:US', 'ACN:US', 'ADBE:US', 'ADDYY:US', 'ADI:US', 'ADM:US', 'ADP:US', 'ADRNY:US', 'ADSK:US', 'AEE:US', 'AEIS:US', 'AEP:US', 'AER:US', 'AES:US', 'AESI:US', 'AFL:US', 'AGCO:US', 'AGIO:US', 'AIG:US', 'AIQUY:US', 'AIT:US', 'AIZ:US', 'AJG:US', 'AKAM:US', 'ALB:US', 'ALC:US', 'ALE:US', 'ALFVY:US', 'ALG:US', 'ALGM:US', 'ALGN:US', 'ALGT:US', 'ALIZY:US', 'ALK:US', 'ALKT:US', 'ALL:US', 'ALLY:US', 'AMAT:US', 'AMCR:US', 'AMD:US', 'AME:US', 'AMGN:US', 'AMP:US', 'AMSF:US', 'AMT:US', 'AMTM:US', 'AMZN:US', 'ANET:US', 'ANSS:US', 'AON:US', 'AOS:US', 'APD:US', 'APG:US', 'APH:US', 'APO:US', 'APP:US', 'APPF:US', 'APTV:US', 'AR:US', 'ARE:US', 'ARES:US', 'ARIS:US', 'ARLP:US', 'ARMK:US', 'ARW:US', 'ASAZY:US', 'ASC:US', 'ATKR:US', 'ATLKY:US', 'ATO:US', 'ATR:US', 'ATRC:US', 'AVB:US', 'AV

In [6]:
#SCRAPING FROM STOCK ANALYSIS
# ------------------------- helper -------------------------
def _parse_sector(html):
    soup  = BeautifulSoup(html, "html.parser")
    label = soup.find("span", string="Sector")
    if label:
        tag = label.find_next("a", class_="dothref text-default")
        if tag:
            return tag.text.strip()
    return "Sector not found"

# ------------------------- main ---------------------------
def get_sector(ticker):
    base = ticker.split(":")[0].lower().replace("/", ".")  # Convert slashes to dots for URL compatibility

    # 1) try exchange-listed stock
    stock_url = f"https://stockanalysis.com/stocks/{base}/"
    try:
        html = requests.get(stock_url, timeout=30).raise_for_status() or requests.get(stock_url, timeout=30).text
        return _parse_sector(html)
    except requests.exceptions.HTTPError as err:
        if err.response is None or err.response.status_code != 404:
            return f"Error: {err}"

    # 2) try ETF  ➜ return None (always unrelated)
    etf_url = f"https://stockanalysis.com/etf/{base}/"
    try:
        requests.get(etf_url, timeout=30).raise_for_status()
        return None
    except requests.exceptions.HTTPError as err:
        if err.response is None or err.response.status_code != 404:
            return f"Error: {err}"

    # 3) try OTC stock
    otc_url = f"https://stockanalysis.com/quote/otc/{base}/"
    try:
        html = requests.get(otc_url, timeout=30).raise_for_status() or requests.get(otc_url, timeout=30).text
        return _parse_sector(html)
    except Exception as e:
        return f"Error: {e}"


# Removing crypto
tickers = unique_tickers[10:]

#Splitting into ticker chunks for scraping timeouts... 1 -> [0:300] , 2 -> [300:600], <TODO LATER>
chunk1_tickers = tickers[0:300]
chunk2_tickers = tickers[283:500] #Starting at 283, since 284 failed.
chunk3_tickers = tickers[462:600] #Same reason as above
chunk4_tickers = tickers[600:800]
chunk5_tickers = tickers[798:]

results = {}
for i, tk in enumerate(chunk5_tickers, 1):
    results[tk] = get_sector(tk)
    if i % 10 == 0:                    # pause every 10 requests
        time.sleep(random.uniform(2, 4))
print(results)


{'SONO:US': 'Technology', 'SONVY:US': 'Healthcare', 'SONY:US': 'Technology', 'SPCE:US': 'Industrials', 'SPDW:US': None, 'SPG:US': 'Real Estate', 'SPGI:US': 'Financials', 'SPHY:US': None, 'SPOT:US': 'Communication Services', 'SPXC:US': 'Industrials', 'SPY:US': None, 'SPYG:US': None, 'SPYV:US': None, 'SQ:US': 'Technology', 'SRE:US': 'Utilities', 'SRPT:US': 'Healthcare', 'SRVR:US': None, 'SSD:US': 'Materials', 'SSNC:US': 'Technology', 'ST:US': 'Technology', 'STAG:US': 'Real Estate', 'STE:US': 'Healthcare', 'STLD:US': 'Materials', 'STRL:US': 'Industrials', 'STT:US': 'Financials', 'STX:US': 'Technology', 'STZ:US': 'Consumer Staples', 'SU:US': 'Energy', 'SUI:US': 'Real Estate', 'SUN:US': 'Energy', 'SUPN:US': 'Healthcare', 'SWK:US': 'Industrials', 'SWKS:US': 'Technology', 'SWTX:US': 'Healthcare', 'SYF:US': 'Financials', 'SYIEY:US': 'Materials', 'SYK:US': 'Healthcare', 'SYY:US': 'Consumer Staples', 'T:US': 'Communication Services', 'TAK:US': 'Healthcare', 'TAP:US': 'Consumer Staples', 'TBLL:US